# SciPy統計テスト チュートリアル Notebook

`README.md` の内容を Jupyter 上で再生し、各サンプルのソースと出力をまとめて確認できます。


## 環境セットアップ
以下のコマンドで uv ベースの仮想環境を用意してから実行してください。
```bash
uv venv .venv
source .venv/bin/activate
uv pip install numpy scipy
```


## 1. 一標本 t 検定
- **対象スクリプト**: `examples/t_test_one_sample.py`
- **目的**: 平均時短が 3.5 分を超えるかどうかを一標本 t 検定で確認します。
- **実行コマンド**: `uv run --with numpy --with scipy python3 examples/t_test_one_sample.py`

### コマンド/オプションのポイント
- `--with numpy` でサンプル配列を即時計算環境へ追加
- `--with scipy` で `stats.ttest_1samp` を提供
- `examples/t_test_one_sample.py` を別ファイルに変えると他シナリオも再利用可能

### スクリプト確認と実行


In [ ]:
from pathlib import Path
import runpy

path = Path('examples/t_test_one_sample.py')
print('--- examples/t_test_one_sample.py ---')
print(path.read_text())
print('
--- 実行結果 ---')
runpy.run_path(str(path))


## 2. Welch の 2 標本 t 検定
- **対象スクリプト**: `examples/t_test_independent.py`
- **目的**: 旧/新オンボーディングの満足度平均差を Welch t 検定で比較します。
- **実行コマンド**: `uv run --with numpy --with scipy python3 examples/t_test_independent.py`

### コマンド/オプションのポイント
- `equal_var=False` で分散が異なる前提でも対応
- `--with numpy` で 2 群の配列演算を支援
- `--with scipy` で `stats.ttest_ind` (Welch) を実行

### スクリプト確認と実行


In [ ]:
from pathlib import Path
import runpy

path = Path('examples/t_test_independent.py')
print('--- examples/t_test_independent.py ---')
print(path.read_text())
print('
--- 実行結果 ---')
runpy.run_path(str(path))


## 3. 対応のある t 検定
- **対象スクリプト**: `examples/t_test_paired.py`
- **目的**: 同一ドライバーの訓練前後で反応時間が短縮したかを確認します。
- **実行コマンド**: `uv run --with numpy --with scipy python3 examples/t_test_paired.py`

### コマンド/オプションのポイント
- `stats.ttest_rel` はペア差分を直接扱う
- `--with numpy` で Before/After 配列を整形
- データ順序を崩さないようスクリプトの配列を維持

### スクリプト確認と実行


In [ ]:
from pathlib import Path
import runpy

path = Path('examples/t_test_paired.py')
print('--- examples/t_test_paired.py ---')
print(path.read_text())
print('
--- 実行結果 ---')
runpy.run_path(str(path))


## 4. カイ二乗適合度検定
- **対象スクリプト**: `examples/chi_square_goodness_of_fit.py`
- **目的**: 曜日別売上が一様分布と一致するかを検定します。
- **実行コマンド**: `uv run --with numpy --with scipy python3 examples/chi_square_goodness_of_fit.py`

### コマンド/オプションのポイント
- `np.repeat` で一様期待度数を構築
- `stats.chisquare` に観測・期待ベクトルを渡す
- カテゴリを増やす際は `observed` を編集

### スクリプト確認と実行


In [ ]:
from pathlib import Path
import runpy

path = Path('examples/chi_square_goodness_of_fit.py')
print('--- examples/chi_square_goodness_of_fit.py ---')
print(path.read_text())
print('
--- 実行結果 ---')
runpy.run_path(str(path))


## 5. カイ二乗独立性の検定
- **対象スクリプト**: `examples/chi_square_independence.py`
- **目的**: 性別と色の嗜好が関連するかを 2×2 分割表で検定します。
- **実行コマンド**: `uv run --with numpy --with scipy python3 examples/chi_square_independence.py`

### コマンド/オプションのポイント
- `stats.chi2_contingency` で統計量・p 値・期待度数を取得
- `--with numpy` で分割表を `np.array` 化
- 列や行を増やす場合は行列サイズを変える

### スクリプト確認と実行


In [ ]:
from pathlib import Path
import runpy

path = Path('examples/chi_square_independence.py')
print('--- examples/chi_square_independence.py ---')
print(path.read_text())
print('
--- 実行結果 ---')
runpy.run_path(str(path))


## 6. カイ二乗同質性の検定
- **対象スクリプト**: `examples/chi_square_homogeneity.py`
- **目的**: 複数クリエイティブ間でコンバージョン率が同じかを比較します。
- **実行コマンド**: `uv run --with numpy --with scipy python3 examples/chi_square_homogeneity.py`

### コマンド/オプションのポイント
- 行がキャンペーン、列が結果（成約/非成約）
- `stats.chi2_contingency` を再利用
- 期待度数で差の要因を把握

### スクリプト確認と実行


In [ ]:
from pathlib import Path
import runpy

path = Path('examples/chi_square_homogeneity.py')
print('--- examples/chi_square_homogeneity.py ---')
print(path.read_text())
print('
--- 実行結果 ---')
runpy.run_path(str(path))
